## New Summarizer experiments1

What I need to do

- Retrieve the today news, article from the scrapper database.

- Summarize them with good prompt.

- display them on a UI.

In [ ]:
%load_ext dotenv

In [ ]:
%dotenv ./.env_prod -o

In [ ]:
from src.shared.database import execute_query, generate_database_connection

In [ ]:
yesterday_article_query = "select content, title, posted_at,url from article where posted_at::date = CURRENT_DATE - interval '1 day'"

In [ ]:
from os import getenv

In [ ]:
database_user = getenv('POSTGRES_USER')
database_password = getenv('POSTGRES_PASSWORD')
database_host = getenv('POSTGRES_HOST')
database_port = getenv('POSTGRES_PORT')
database_name = getenv('POSTGRES_DB')

In [ ]:
database_credentials = {
    'user': database_user,
    'password': database_password,
    'host': database_host,
    'port': database_port,
    'database': database_name
}

In [ ]:
connection = generate_database_connection(database_crendentials=database_credentials)

In [ ]:
connection

In [ ]:
results =execute_query(query=yesterday_article_query, database_connection=connection)

In [ ]:
results[0].url

In [ ]:
import pandas as pd

In [ ]:
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df.columns = ["content", "title", "posted_at", "url"]

In [ ]:
from pathlib import Path

In [ ]:
current_directory = Path.cwd().joinpath("datasets", "today_news")

In [ ]:
current_directory.mkdir(exist_ok=True)

In [ ]:
from datetime import datetime

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")

In [ ]:
results_df.to_csv(current_directory.joinpath(f"{today}-news.csv"))

In [ ]:
results_df.content

In [ ]:
results_df = results_df.drop_duplicates(subset="content")

In [ ]:
results_df.head()

In [ ]:
from src.rag.components.generator import LLamaCppGeneratorComponent

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
prompt = "You are a french news reporter"

In [ ]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [ ]:
llama_cpp_generator._ping_api()

prompting doesn't seem to wokr, let us use clustering and prompting.

The step will be computing the embedding of the documents, and then cluster the document. Then for each cluster of documents, we will generate a summary.

embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
from pathlib import Path

In [ ]:
model_path = Path.cwd().joinpath("models")

In [ ]:
embedding_model_path = model_path.joinpath(embedding_model_id)

In [ ]:

transformer_kwargs = {"model_name_or_path": embedding_model_path.__str__(),
                      "trust_remote_code": True,
                      "device": "cpu",
                      "config_kwargs": {"use_memory_efficient_attention": False,
                                        "unpad_inputs": False},
                      "cache_folder": model_path}

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sentence_transformer_model = SentenceTransformer(
    **transformer_kwargs)

In [ ]:
today_news_embeddings = sentence_transformer_model.encode(
    results_df.content.tolist())

Not just clustering but let start with paragraph spliting and then embedding.

In [ ]:
today_news_embeddings

In [ ]:
today_news_embeddings.shape

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
axes = plt.figure(figsize=(5, 10))

In [ ]:
similarities = sentence_transformer_model.similarity(today_news_embeddings, today_news_embeddings)

In [ ]:
import seaborn as sb

In [ ]:
#plt.figure(figsize=(50, 50))
#sb.heatmap(similarities, cmap="Blues", annot=True)

In [ ]:
results_df.iloc[25]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

In [ ]:
def find_best_estimator (X):
    K_mean_estimators = [
        (f"KMeans_{i}", KMeans(n_clusters=i, random_state=42)) for i in [ 5, 7, 10, 12]]

    best_estimator = None
    best_metric = float("-inf")
    for estimator_name, estimator in K_mean_estimators :
        estimator.fit(X)
        labels = estimator.labels_
        score = silhouette_score(
            today_news_embeddings, labels, metric="cosine")
        if score > best_metric :
            best_metric = score
            best_estimator = estimator
        print(estimator_name, score)
    return best_estimator

In [ ]:
A value of 1,

In [ ]:
best_estimator = find_best_estimator(today_news_embeddings)

In [ ]:
best_estimator

In [ ]:
results_df["labels"] = best_estimator.labels_

In [ ]:
results_df.query("labels ==11")

In [ ]:
summarization_prompt = """
Give a summary in french of the following document:
{{content}}
 Describes it in a style of a french new paper reporters.

Don't summarize each document separately, the content in all the documents should be summarized.

The summary should be in french not in English
"""

In [ ]:
first_cluster = results_df.query("labels == 5")
first_cluster

In [ ]:
first_cluster.content.str.cat(sep="\n")

In [ ]:
llama_cpp_generator.generate_chat_input(
    template_values={"content": first_cluster.content.str.cat(sep="\n")}, prompt_template=summarization_prompt)

In [ ]:
summaries = []
for id, news_group in results_df.groupby("labels"):
    news = news_group.content.str.cat(sep="\n")
    summary = llama_cpp_generator.run(template_values={"content": news}, prompt_template=summarization_prompt)
    summaries.append(summary)
    print(summary)
    print("---" * 10)

I will have to stop here, and then start writing about the learning. 

- Another apporach will be collect the document from radiookapi and fine tune the model.
-   Deploy the solution with streamlit

### Trying Topic Modeling and Then Summarizing

In  this section we will use topic modelling and then summarize the documents in each topic.

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

In [ ]:
from spacy.lang.fr.stop_words import STOP_WORDS as french_stop_words

In [ ]:
CONGO_WORDS = {
    'congo',
    'congolais',
    "congolaise"
    'rdc',
    'drc',
    '-',
    'https',
    'rdcongo',
    'drc',
    'drcongo'}

In [ ]:
words_to_remove = CONGO_WORDS.union(french_stop_words)

In [ ]:
hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean',
                        cluster_selection_method='eom', prediction_data=True)

In [ ]:
umap_model = UMAP(n_neighbors=15, n_components=15,
                  min_dist=0.0, metric='cosine')

In [ ]:
count_vectorizer = CountVectorizer(
    stop_words=list(words_to_remove), ngram_range=(1, 2))

In [ ]:
ctfidf_model = ClassTfidfTransformer()

In [ ]:
representation_model = KeyBERTInspired()

In [ ]:
topic_model = BERTopic(
    embedding_model=sentence_transformer_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=best_estimator,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=count_vectorizer,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    # Step 6 - (Optional) Fine-tune topic representations
    representation_model=representation_model
)

In [ ]:
topics, probs = topic_model.fit_transform(results_df.content)

In [ ]:
probs

In [ ]:
topic_model.visualize_topics()

In [ ]:
new_topics  = topic_model.reduce_topics(
    results_df.content, nr_topics=7)


In [ ]:
new_topics.visualize_topics()

In [ ]:
topic_model.visualize_documents(results_df.content, embeddings=reduced_embebddings)

In [ ]:
find_best_estimator(reduced_embeddings)

The bert topic modeling is not really topic model, it is mostly document reprenstaiton plus document clustering. So if we finetune our clustering algorithm better we can get better results.

### Analyzing inter cluster Similarities.

In [ ]:
results_df.head()

In [ ]:
document_index = results_df.query("labels == 1").index

In [ ]:
vectors = today_news_embeddings[document_index]

In [ ]:
sentence_transformer_model.similarity(vectors ,  vectors)

In [ ]:
def analyse_embeddings(dataframe, embeddings, index, label_column="labels"):
    """ take an matrix of embeddings, and the labels.
    for each label compute the cosine similarity of document with that label.
    """
    document_in_index = dataframe.query(f"{label_column} == {index}")
    with pd.option_context('display.max_colwidth', None):
        display(document_in_index.title)
    document_index = document_in_index.index
    vectors = embeddings[document_index]
    return sentence_transformer_model.similarity(vectors ,  vectors)

    

In [ ]:
similarity_matrix = analyse_embeddings(results_df, today_news_embeddings, 4)

In [ ]:
similarity_matrix

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db_scan_model.fit(today_news_embeddings)

In [ ]:
db_scan_model.labels_

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
sentence_transformer_model.similarity(today_news_embeddings[0], today_news_embeddings[0])[0].item()

In [ ]:
def transformer_cosine_similarity(X, Y):
    return sentence_transformer_model.similarity(X, Y).item()

In [ ]:
db_scan_model = DBSCAN(eps=0.3, min_samples=5, metric="cosine", algorithm="auto")

In [ ]:
db_scan_model.fit(today_news_embeddings)

In [ ]:
db_scan_model.labels_

In [ ]:
results_df["labels_db_scan"] = db_scan_model.labels_

In [ ]:
### Ned to explore nearest neibords models.

Need to explore why db scan is not working, and then work out the nearest neibor stuff

 Another approach to try to get the best document actually should be trying to cluster document based on their titles.

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
neighbors_model = NearestNeighbors(
    radius=0,
    algorithm="auto",
    leaf_size=30,
    metric="cosine",
    metric_params={},
)

In [ ]:
neighbors_model.fit(today_news_embeddings)

In [ ]:
today_news_embeddings[0].reshape(1, -1)

In [ ]:
distances, indices = neighbors_model.kneighbors(today_news_embeddings[10].reshape(1, -1))

In [ ]:
neighbors_distances, neighbor_indices = neighbors_model.radius_neighbors(today_news_embeddings, radius=0.3, return_distance=True)

In [ ]:
neighbor_indices[10]

In [ ]:
neighbors_distances[10]

In [ ]:
indices[0]

In [ ]:
results_df.query("labels == 0")

In [ ]:
analyse_embeddings(results_df, today_news_embeddings, -1, "labels_db_scan")

In [ ]:
today_news_embeddings[indices[0]].shape

In [ ]:
similarities = sentence_transformer_model.similarity(
    today_news_embeddings[indices[0]], today_news_embeddings[indices[0]])

In [ ]:
1 - similarities

Kmean is not giving good results because some cluster are not correct., Let try with the title embeddings.